In [14]:
from pathlib import Path
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torchvision import transforms as T
import torch.nn as nn
import torch
import torch.nn.functional as F
from sklearn.model_selection import GroupKFold
import numpy as np
from fastprogress.fastprogress import master_bar, progress_bar
from sklearn.metrics import accuracy_score, roc_auc_score
from efficientnet_pytorch import EfficientNet
from torchvision import models
import pdb
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt
import pickle
import imgaug

In [15]:
path = Path('input')

# Dataset Initialisation
Here we will construct a class denoting the dataset, and create several methods within it to eliminate some repetitiveness from the code.

In [16]:
class MelanomaDataset(Dataset):
    def __init__(self, df, img_path_one, transforms=None, is_test=False):
        self.df = df
        self.img_path_one = img_path_one
        self.transforms = transforms
        self.is_test = is_test
        
    def __getitem__(self, indx):
        img_path = f"{self.img_path_one}/{self.df.iloc[indx]['image_name']}.jpg"
        img = Image.open(img_path)

        if self.transforms:
            img = self.transforms(**{"image": np.array(img)})["image"]
            
        if self.is_test:
            return img

        target = self.df.iloc[indx]['target']
        return img, torch.tensor([target], dtype=torch.float32)
    
    def __len__(self):
        return self.df.shape[0]

# Data Augmentation
Data augmentation is the process of increasing the amount of data available for training, and is especially relevant here as it can increase the accuracy of the model. This in turn makes sure that the patient/doctor receives a more accurate result — in the medical industry, every percent matters. An added benefit is that these alterations can make the model adapt better to images in 'strange' conditions (e.g. bad lighting, low resolution camera, wrong amount of zoom).

In [17]:
def get_augmentations(p=0.5):

    # Assigning the ImageNet mean and standard deviation to a variable
    imagenet_stats = {'mean':[0.485, 0.456, 0.406], 'std':[0.229, 0.224, 0.225]}

    # Using albumentations' augmentation pipeline to sequentially alter the images.
    # The variable p stands for the percentage chance of a particular alteration happening.
    train_tf = A.Compose([
        A.Cutout(p=p),
        A.RandomRotate90(p=p),
        A.Flip(p=p),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2,
                                       contrast_limit=0.2,
                                       ),
            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=50,
                val_shift_limit=50)
        ], p=p),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=p),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=p),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=0.1)
        ], p=p), 
        ToTensor(normalize=imagenet_stats)
        ])
    
    # The test dataset does not require augmentation
    test_tf = A.Compose([
        ToTensor(normalize=imagenet_stats)
        ])

    
    return train_tf, test_tf

# Train / Validation Split
- Any duplicate images must be removed
- In accordance with Chris Deotte's triple-stratified K-Fold [model](https://www.kaggle.com/code/cdeotte/triple-stratified-kfold-with-tfrecords), the split will be conducted in a 80:20 ratio

In [18]:
def get_train_val_split(df):
    # Removal of duplicates
    df = df[df.tfrecord != -1].reset_index(drop=True)

    # Splitting
    train_tf_records = list(range(len(df.tfrecord.unique())))[:12]
    split_cond = df.tfrecord.apply(lambda x: x in train_tf_records)
    train = df[split_cond].reset_index()
    valid = df[~split_cond].reset_index()
    return train, valid

# Model Initialisation
- EfficientNet is a convolutional neural network architecture and scaling method that uniformly scales all dimensions of depth/width/resolution using a compound coefficient. 
- This makes it ideal for this project — by tinkering with which model is used (B0 until B7, with each model sequentially using more system resources and being more accurate)

In [19]:
class Model(nn.Module):
    def __init__(self, model_name='efficientnet-b0', pool=F.adaptive_avg_pool2d):
        super().__init__()
        self.pool = pool
        self.backbone = EfficientNet.from_pretrained(model_name)
        in_features = getattr(self.backbone,'_fc').in_features
        self.classifier = nn.Linear(in_features,1)


    def forward(self, x):
        features = self.pool(self.backbone.extract_features(x),1)
        features = features.view(x.size(0),-1)
        return self.classifier(features)

# Helper Functions

In [20]:
# Change the device to "cuda" if you have a GPU or "cpu" if you if you don't. Even mps is supported on Apple Silicon, it is not as fast as cpu.
device = torch.device("cpu")

def get_model(model_name='efficientnet-b0', lr=1e-5, wd=0.01, freeze_backbone=False, opt_fn=torch.optim.AdamW, device=None):
    model = Model(model_name=model_name)

    # Freezing Layers
    if freeze_backbone:
        for parameter in model.backbone.parameters():
            parameter.requires_grad = False
    
    # Optimising weights
    opt = opt_fn(model.parameters(), lr=lr, weight_decay=wd)
    model = model.to(device)
    return model, opt

# Training
def training(xb, yb, model, loss_fn, opt, device, scheduler):
    xb, yb = xb.to(device), yb.to(device)
    out = model(xb)
    opt.zero_grad()
    loss = loss_fn(out,yb)
    loss.backward()
    opt.step()
    scheduler.step()
    return loss.item()

# Validation
def validation(xb,yb,model,loss_fn,device):
    xb,yb = xb.to(device), yb.to(device)
    out = model(xb)
    loss = loss_fn(out,yb)
    out = torch.sigmoid(out)
    return loss.item(),out

def get_data(train_df, valid_df, train_tfms, test_tfms, bs):
    train_ds = MelanomaDataset(df=train_df, img_path_one=path/'train', transforms=train_tfms)
    valid_ds = MelanomaDataset(df=valid_df, img_path_one=path/'train', transforms=test_tfms)
    train_dl = DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, num_workers=4)
    valid_dl = DataLoader(dataset=valid_ds, batch_size=bs*2, shuffle=False, num_workers=4)
    return train_dl, valid_dl

def fit(epochs, model, train_dl, valid_dl, opt, devic=None, loss_fn=F.binary_cross_entropy_with_logits):
    devic = device
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, len(train_dl)*epochs)
    val_rocs = [] 
    
    #Creating progress bar
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','val_roc'],table=True)

    for epoch in mb:    
        trn_loss,val_loss = 0.0,0.0
        val_preds = np.zeros((len(valid_dl.dataset),1))
        val_targs = np.zeros((len(valid_dl.dataset),1))
        
        #Training
        model.train()
        
        #For every batch 
        for xb,yb in progress_bar(train_dl,parent=mb):
            trn_loss += training(xb,yb,model,loss_fn,opt,devic,scheduler) 
        trn_loss /= mb.child.total

        #Validation
        model.eval()
        with torch.no_grad():
            for i,(xb,yb) in enumerate(progress_bar(valid_dl,parent=mb)):
                loss,out = validation(xb, yb, model, loss_fn, devic)
                val_loss += loss
                bs = xb.shape[0]
                val_preds[i*bs:i*bs+bs] = out.cpu().numpy()
                val_targs[i*bs:i*bs+bs] = yb.cpu().numpy()

        val_loss /= mb.child.total
        val_roc = roc_auc_score(val_targs.reshape(-1),val_preds.reshape(-1))
        val_rocs.append(val_roc)

        mb.write([epoch,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{val_roc:.6f}'],table=True)
    return model,val_rocs

# Training

In [21]:
df = pd.read_csv(path/'train.csv')
train_df, valid_df = get_train_val_split(df)
train_tfms, test_tfms = get_augmentations(p=0.5)
train_dl, valid_dl = get_data(train_df, valid_df, train_tfms, test_tfms, 16)
model, opt = get_model(model_name='efficientnet-b0', lr=1e-4, wd=1e-4)
model, val_rocs = fit(8, model, train_dl, valid_dl, opt)
torch.save(model.state_dict(), f'effb0.pth')

Loaded pretrained weights for efficientnet-b0


Training does not work in Jupyter Notebooks, so refer to train.py to see the whole training process.

# Testing

### EfficientNet B0, 1 TTA Iteration

In [ ]:
# All the code below does not work due to the Jupyter Notebooks environment. Please refer to train.py instead!
model, opt = get_model(model_name='efficientnet-b0',lr=1e-4,wd=1e-4)
model.load_state_dict(torch.load(f'effb0.pth', map_location=device))
model.eval()
test_df = pd.read_csv(path/'test.csv')
test_ds = MelanomaDataset(df=test_df, img_path_one=path/'test',transforms=test_tfms, is_test=True)
test_dl = DataLoader(dataset=test_ds, batch_size=32, shuffle=False,num_workers=4)

tta = 1
preds = np.zeros(len(test_ds))
for tta_id in range(tta):
    count = 0
    test_preds = []
    with torch.no_grad():
        for xb in test_dl:
            print(count)
            xb = xb.to(device)
            out = model.to(device)(xb)
            out = torch.sigmoid(out)
            test_preds.extend(out.cpu().detach().numpy())
            count += 1
        preds += np.array(test_preds).reshape(-1)
    print(f'TTA {tta_id+1}')
preds /= tta

subm = pd.read_csv(path/'sample_submission.csv')
subm.target = preds
subm.to_csv('submissions/submission.csv', index=False)

With only one TTA iteration, this model was able to achieve a 0.8868 area under the ROC Curve:
![Score](img/effb0tta1.png)

### EfficientNet B0, 10 TTA Iterations

![Score](img/effb0tta10.png)